In [1]:
from pynq import Overlay
import time
import matplotlib.pyplot as plt
import numpy as np

ol = Overlay("/home/xilinx/please_work/working.bit")

In [2]:
oux = ol.u_x.channel1
ouy = ol.u_y.channel1
orho = ol.rho.channel1
istep = ol.step.channel1
ireset_lbm = ol.reset_lbm.channel1
istart_transmit = ol.start_transmit.channel1

In [21]:
# reset all
ireset_lbm.write(0,1) #reset
print(f"reset state: {ireset_lbm.read()}") # check reset
time.sleep(1) # wait
ireset_lbm.write(1,1) #reset
print(f"reset state: {ireset_lbm.read()}") # check non-reset

# check start transmit
istart_transmit.write(0,-1)
print(f"transmit state: {istart_transmit.read()}")

reset state: 0
reset state: 1
transmit state: 0


In [22]:
for step in range(0,1000,100):
    istep.write(step,0xFFFF)
#     print(f"current step: {istep.read()}")
    time.sleep(0.1) # give it way more time than it needs
    uxs = []
    rhos = []
    istart_transmit.write(0x8000,0x8000) # raise query state flag
    for pixel in range(2500):
        istart_transmit.write(pixel,0x0FFF) # get pixel
#             print(f"transmit state: {istart_transmit.read()}")
        rhos.append(float(orho.read())/(2**13))
#             time.sleep(0.0005) # give it time
    plt.imsave(f"imgs/step{step}.png", np.reshape(rhos,(50,50)))

In [23]:
import shutil
shutil.make_archive("img_zip", 'zip', "imgs")

'/home/xilinx/jupyter_notebooks/img_zip.zip'